# Generator - Approach 1

In [34]:
import json

from matplotlib.font_manager import json_load
import readNews
import sentimentAnalysis

### Read the news

In [35]:
def clean (ListSentences):

    l = [s.replace("\"","'") for s in ListSentences]

    return l

In [45]:

url = "https://www.snopes.com/fact-check/strollers-poland-ukraine-refugees/"
summary = readNews.main(url, "summary").encode('utf8').decode().split('\n')
title = readNews.main(url, "title").encode('utf8').decode().split('\n')
newsSentences = title + summary
text = readNews.main(url, "text").encode('utf8').decode().split('\n')



In [46]:
newsSentences


['Were Strollers Left at Poland Train Station for Ukraine Refugees?',
 'A picture showing a line of seven strollers at a train station was widely shared on social media with the caption: “Parents in Poland left their baby strollers in rail road stations for the Ukrainian moms that fled carrying their children.”This appears to be a genuine photograph that truly depicts a set of strollers that were left for Ukrainian refugees.',
 'This photograph was taken by Francesco Malavolta, a photojournalist covering the conflict in Ukraine.',
 '– Recently many people have come to us from Odessa, the next train will come from Lviv – says Janusz Łukasiewicz, station commander on Polish Radio 1.',
 'Refugees get a warm meal, the most necessary things and a place where they can rest before their onward journey.',
 'Here are some additional photographs of the scenes at this Polish train station.']

In [47]:
text

['In March 2022, as Ukrainians fled their home country after Russia’s invasion, a photograph started to circulate on social media that supposedly showed an act of kindness from one of Ukraine’s neighbors, Poland.',
 '',
 'A picture showing a line of seven strollers at a train station was widely shared on social media with the caption: “Parents in Poland left their baby strollers in rail road stations for the Ukrainian moms that fled carrying their children.”',
 '',
 'This appears to be a genuine photograph that truly depicts a set of strollers that were left for Ukrainian refugees.',
 '',
 'This photograph was taken by Francesco Malavolta, a photojournalist covering the conflict in Ukraine. Malavolta posted this photo to his Twitter account on March 3, along with the caption “Passeggini lasciati in stazione per le donne in arrivo dall’Ucraina con neonati,” which translates from Italian to “Strollers left at the station for women arriving from Ukraine with babies.”',
 '',
 'Here’s the o

#### Sentiment Analysis

In [26]:

text =clean(newsSentences)
lang = 'pt'
sentiments = sentimentAnalysis.main(text,lang)


In [27]:
## Ordenar por ordem decrescente maior emoçao
playload = json.loads(sentiments)

In [28]:
type(playload)

dict

In [29]:

## Ordenar as frases por aquela que tem maior surpresa
sortedByEmotion = {k: v for k, v in sorted(playload.items(), key=lambda item: (item[1]['emotions']['sadness'], item[1]['emotions']['surprise']), reverse=True)}


In [30]:
sortedByEmotion

{'As of this writing, it had been upvoted more than 4,400 times.': {'emotions': {'anger': 0.0,
   'anticipation': 0.0,
   'disgust': 0.0,
   'fear': 0.0,
   'joy': 0.0,
   'sadness': 0.0,
   'surprise': 0.0,
   'trust': 0.0},
  'polarities': {'negative': 0.0, 'positive': 0.0}},
 'Did AOC Say This About Trump?': {'emotions': {'anger': 0.0,
   'anticipation': 0.0,
   'disgust': 0.0,
   'fear': 0.0,
   'joy': 0.0,
   'sadness': 0.0,
   'surprise': 0.0,
   'trust': 0.0},
  'polarities': {'negative': 0.0, 'positive': 0.0}},
 'Happy reading and let us know what else you might be interested in knowing.': {'emotions': {'anger': 0.0,
   'anticipation': 0.0,
   'disgust': 0.0,
   'fear': 0.0,
   'joy': 0.0,
   'sadness': 0.0,
   'surprise': 0.0,
   'trust': 0.0},
  'polarities': {'negative': 0.0, 'positive': 0.0}},
 'In March 2022, a quote from U.S. Rep. Alexandria Ocasio-Cortez, a democrat from New York, resurfaced after a now-removed Reddit post shared a screenshot of the Tweet to the thread r

## Text summarization


All the models present next are pre trained and presents at transformer package given by Huggingface

### Pipeline API

In [39]:
## Importar as biblitecas necessarias

from transformers import pipeline


In [40]:

# using pipeline API for summarization task
summarization = pipeline("summarization",max_length=265)

original_text = ' '.join(text)

summary_text = summarization(original_text)[0]['summary_text']
print("Summary:", summary_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Summary:  In a tweet shared on Oct. 22, 2020, U.S. Rep. Alexandria Ocasio-Cortez said: “If someone told me 3 years ago, while I was waitressing to help my family stay afloat, that in a few short years an unhinged President of the United States would be repeatedly saying my name at the 2020 debate, I would’ve brought them some water and told them to sober up.” A now-removed Reddit post shared a screenshot of the Tweet to the thread r/facepalm .


Summary:  A photograph of a woman holding a weapon was widely circulated on social media as if it showed former Miss Ukraine Anastasiia Lenna taking up arms after Russia’s invasion of Ukraine in early 2022 . Lenna posted a photograph of herself holding her rifle along with the hashtags #standwithukraine and #handsoffukraine . This led many to make the assumption that Lenna had actually joined the Ukrainian army . When the picture went viral, Lenna explained that she wasn’t actually in the military and that her post was just meant to show support Ukrainian forces .

### T5 Model

https://www.thepythoncode.com/article/text-summarization-using-huggingface-transformers-python


In [54]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [55]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")


In [56]:
original_text = ' '.join(text)

In [59]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("summarize: " + original_text, return_tensors="pt", max_length=150, truncation=True)

In [60]:
# generate the summarization output
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,     3,     9,  6419,  2924,     3,     9,   689,    13,  2391,
         13593,   277,    44,     3,     9,  2412,  2478,    47,  5456,  2471,
            30,   569,   783,     3,     5,    48,  3475,    12,    36,     3,
             9,  7746,  6419,    24,  1892,  9850,     7,     3,     9,   356,
            13, 13593,   277,   646,    21, 22849, 16723,     3,     5,    48,
          6419,    47,  1026,    57,     3,     9,  1202, 25340,   343,  6013,
             8,  4129,    16, 11897,     3,     5,     1]])
<pad> a photograph showing a line of seven strollers at a train station was widely shared on social media. this appears to be a genuine photograph that truly depicts a set of strollers left for Ukrainian refugees. this photograph was taken by a photojournalist covering the conflict in Ukraine.</s>


### BERT Model

In [2]:
from summarizer import Summarizer

ImportError: cannot import name 'Summarizer' from 'summarizer' (unknown location)

In [49]:
original_text = ' '.join(text)

In [52]:
#Create default summarizer model
# By default bert-extractive-summarizer uses the ‘bert-large-uncased‘ pretrained model.
model = Summarizer()

summary = model(original_text, max_length=200, ratio=0.01)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
summary

'This appears to be a genuine photograph that truly depicts a set of strollers that were left for Ukrainian refugees. Many of them do not know languages \u200b\u200bother than Russian and Ukrainian, some of them are abroad for the first time.'

In [ ]:
sentiments = sentimentAnalysis.main(summary,lang)
playload = json.loads(sentiments)

sortedByEmotion = {k: v for k, v in sorted(playload.items(), key=lambda item: (item[1]['emotions']['sadness'], item[1]['emotions']['surprise']), reverse=True)}
sortedByEmotion

## Keyword extraction

In [6]:
from keybert import KeyBERT

### https://pypi.org/project/keybert/

2022-03-07 14:02:56.993032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-07 14:02:56.993087: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
original_text = ' '.join(text)

In [13]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(original_text, keyphrase_ngram_range=(1, 1))

In [14]:
keywords

[('fictional', 0.3562),
 ('news', 0.3514),
 ('satire', 0.3411),
 ('conspiracy', 0.3363),
 ('putin', 0.3152)]

In [10]:
original_text

'In March 2022, as Russia continued its attack on Ukraine, the website “Real Raw News” published a fictitious article that falsely claimed a villa owned by U.S. President Joe Biden was bombed by the Russian military.  There is no truth to this story. The website Real Raw News does not traffic in genuine news stories. Rather, this site hosts a series of fictional pieces that play on right-wing conspiracy theories.  A disclaimer on Real Raw News states that the website “contains humor, parody, and satire.”  While Real Raw News carries a disclaimer, its content is often spread on social media or republished on other websites without any mention of its fictional origins. The aforementioned article, for example, was presented as genuine news on TheTruthPatriots.com and on the Diannemarshallreport.com.  The Debunked ‘U.S. Biolab’ Theory  In the days after Russia’s invasion of Ukraine, a narrative started circulating on social media that Putin’s attack was focused on a series of secret biolab